In [49]:
import kquant as kq
from tqdm import tqdm

In [50]:
# set api account
kq.set_api("KRX2308020", "EQDkUcyI3dK6oIAXqAR8BXOK4bKxHHmH")

In [42]:
# get using symbol

symbols_raw_df = kq.symbol_stock()
symbols_df = symbols_raw_df.copy()

filtered_symbols_df = symbols_df[
    (symbols_df["MARKET"].isin(["코스닥", "유가증권"]))
    & (symbols_df["ADMIN_ISSUE"] == 0)
    & (symbols_df["SEC_TYPE"].isin(["ST", "EF", "EN"]))
]

symbols = sorted(set(filtered_symbols_df["SYMBOL"]))

In [43]:
# get stock_data of symbols

stock_df_list = list()

for symbol in tqdm(symbols[:50]):
    _stock_df = kq.daily_stock(symbol)
    stock_df_list.append(_stock_df)

stock_raw_df = pd.concat(stock_df_list, axis=0)

stock_df = stock_raw_df.copy()

100%|██████████| 50/50 [00:24<00:00,  2.04it/s]


In [44]:
# drop zero data row
columns = ["VOLUME", "OPEN", "HIGH", "LOW", "CLOSE"]
for column in columns:
    stock_df = stock_df[stock_df[column] != 0]

In [45]:
# get data only all row are possible
stock_mode_cnt = stock_df.groupby("SYMBOL").count()["CLOSE"].mode()[0]

stock_cnt_series = stock_df.groupby("SYMBOL").count()["CLOSE"]
cnt_filtered_series = stock_cnt_series[stock_cnt_series == stock_mode_cnt]
cnt_filtered_stocks = cnt_filtered_series.index

main_stock_df = stock_df[stock_df["SYMBOL"].isin(cnt_filtered_stocks)]

In [46]:
# get using symbols
symbols = sorted(set(main_stock_df["SYMBOL"]))

In [51]:
stock_info_df = kq.rank_stocks()

## Sigular Symbol

In [35]:
symbol = symbols[0]

In [51]:
_stock_df = main_stock_df[main_stock_df["SYMBOL"] == symbol].copy()
_stock_df.sort_values("DATE", inplace=True)
_stock_df["PRICE_DIFF"] = (_stock_df["OPEN"] - _stock_df["CLOSE"]) / _stock_df["OPEN"]

_price_diff_array_list = _stock_df["PRICE_DIFF"].values

In [52]:
CFG = {
    "dataset_window": 200,
    "input_window": 20,
    "output_window": 15,
}

In [53]:
_price_diff_array_list = _price_diff_array_list[
    -(CFG["dataset_window"] + CFG["input_window"]) :
]

In [54]:
i_window = CFG["input_window"]
o_window = CFG["output_window"]

x_dataset = list()
y_dataset = list()

for idx in range(len(_price_diff_array_list) - i_window - o_window + 1):
    _x = _price_diff_array_list[idx : idx + i_window]
    _y = _price_diff_array_list[idx + i_window : idx + i_window + o_window]
    x_dataset.append(_x)
    y_dataset.append(_y)

x_dataset = np.array(x_dataset)

y_dataset = np.array(y_dataset)
y_dataset = y_dataset.sum(axis=1)

In [57]:
final_x = _price_diff_array_list[-CFG["input_window"] :]

In [10]:
# Model
## Get similairty_score
def get_cosine_similarity(array_1, array_2):
    cosine_similarity = np.dot(array_1, array_2) / (
        np.linalg.norm(array_1) * np.linalg.norm(array_2)
    )
    return cosine_similarity


def get_similarity_df(x_dataset, y_dataset, final_x):
    similarity_results = list()
    for x_data, y_data in zip(x_dataset, y_dataset):
        _similarity_score = get_cosine_similarity(x_data, final_x)
        similarity_results.append(
            {
                "similarity_score": _similarity_score,
                "actual_y": y_data,
            }
        )
    similarity_df = pd.DataFrame(similarity_results)
    return similarity_df


def get_similarity_main_df(x_dataset, y_dataset, final_x, n):
    similarity_results = list()
    for x_data, y_data in zip(x_dataset, y_dataset):
        _similarity_score = get_cosine_similarity(x_data, final_x)
        similarity_results.append(
            {
                "similarity_score": _similarity_score,
                "actual_y": y_data,
            }
        )
    similarity_df = pd.DataFrame(similarity_results)
    similarity_main_df = similarity_df.nlargest(n, "similarity_score")
    return similarity_main_df


def get_pred_y(similarity_df):
    pred_y = (similarity_df["similarity_score"] * similarity_df["actual_y"]).mean()
    return pred_y

In [59]:
similarity_main_df = get_similarity_main_df(x_dataset, y_dataset, final_x, 5)

In [60]:
similarity_main_df

,similarity_score,actual_y
119,0.5126,-0.1572
36,0.4607,0.0221
173,0.4532,0.0332


## Plural SYmbols


In [30]:
CFG = {
    "dataset_window": 200,
    "input_window": 10,
    "output_window": 5,
}

In [31]:
symbol_pred_dict = dict()

for symbol in symbols:
    _stock_df = main_stock_df[main_stock_df["SYMBOL"] == symbol].copy()
    _stock_df.sort_values("DATE", inplace=True)
    _stock_df["PRICE_DIFF"] = (_stock_df["OPEN"] - _stock_df["CLOSE"]) / _stock_df[
        "OPEN"
    ]

    _price_diff_array_list = _stock_df["PRICE_DIFF"].values

    _price_diff_array_list = _price_diff_array_list[
        -(CFG["dataset_window"] + CFG["input_window"]) :
    ]

    i_window = CFG["input_window"]
    o_window = CFG["output_window"]

    x_dataset = list()
    y_dataset = list()

    for idx in range(len(_price_diff_array_list) - i_window - o_window + 1):
        _x = _price_diff_array_list[idx : idx + i_window]
        _y = _price_diff_array_list[idx + i_window : idx + i_window + o_window]
        x_dataset.append(_x)
        y_dataset.append(_y)

    x_dataset = np.array(x_dataset)

    y_dataset = np.array(y_dataset)
    y_dataset = y_dataset.sum(axis=1)

    final_x = _price_diff_array_list[-CFG["input_window"] :]

    similarity_main_df = get_similarity_main_df(x_dataset, y_dataset, final_x, 5)
    pred_y = get_pred_y(similarity_main_df)
    symbol_pred_dict[symbol] = pred_y

In [32]:
result_df=pd.DataFrame.from_dict(symbol_pred_dict,orient='index').reset_index()

In [33]:
result_df.columns = ['SYMBOL','PREDICT']

In [34]:
result_df.sort_values('PREDICT')

,SYMBOL,PREDICT
20,000230,-0.0556
21,000240,-0.0259
26,000325,-0.0240
22,000250,-0.0229
43,000810,-0.0141
7,000100,-0.0140
32,000500,-0.0138
23,000270,-0.0132
45,000850,-0.0123
34,000540,-0.0103


In [35]:
symbols_df[symbols_df["SYMBOL"] == "000215"]

,ISIN,SYMBOL,NAME,NAME_EN,SEC_TYPE,ADMIN_ISSUE,MARKET
24,KR7000211003,000215,DL우,DL(1P),ST,0,유가증권


In [36]:
stock_df[stock_df['SYMBOL'] == "000215"]

,DATE,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,AMOUNT,CHG,CHG_PCT,CHG_TYPE,EX_EVENT,OPEN_NOTADJ,HIGH_NOTADJ,LOW_NOTADJ,CLOSE_NOTADJ,CHG_NOTADJ,CHG_PCT_NOTADJ,CHG_TYPE_NOTADJ,MARKETCAP,SHARES
0,2022-09-07,000215,"38,200","38,700","38,000","38,700",387,"14,771,250",0,0.0000,3,00,"38,200","38,700","38,000","38,700",0,0.0000,3,"65,252,650,500","1,686,115"
1,2022-09-08,000215,"38,700","38,700","37,800","38,100","1,229","46,754,250",-600,-1.5500,5,00,"38,700","38,700","37,800","38,100",-600,-1.5500,5,"64,240,981,500","1,686,115"
2,2022-09-13,000215,"38,700","39,800","38,600","39,500","1,926","74,807,150","1,400",3.6700,2,00,"38,700","39,800","38,600","39,500","1,400",3.6700,2,"66,601,542,500","1,686,115"
3,2022-09-14,000215,"38,350","39,350","38,350","39,300",57,"2,224,700",-200,-0.5100,5,00,"38,350","39,350","38,350","39,300",-200,-0.5100,5,"66,264,319,500","1,686,115"
4,2022-09-15,000215,"39,350","39,350","38,500","38,900",597,"23,087,350",-400,-1.0200,5,00,"39,350","39,350","38,500","38,900",-400,-1.0200,5,"65,589,873,500","1,686,115"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2023-09-01,000215,"19,850","19,850","19,810","19,850",659,"13,079,870",0,0.0000,3,00,"19,850","19,850","19,810","19,850",0,0.0000,3,"33,469,382,750","1,686,115"
245,2023-09-04,000215,"19,750","19,830","19,620","19,830","1,207","23,764,130",-20,-0.1000,5,00,"19,750","19,830","19,620","19,830",-20,-0.1000,5,"33,435,660,450","1,686,115"
246,2023-09-05,000215,"19,850","19,850","19,500","19,850","2,109","41,353,050",20,0.1000,2,00,"19,850","19,850","19,500","19,850",20,0.1000,2,"33,469,382,750","1,686,115"
247,2023-09-06,000215,"19,840","19,840","19,470","19,600","4,180","81,928,560",-250,-1.2600,5,00,"19,840","19,840","19,470","19,600",-250,-1.2600,5,"33,047,854,000","1,686,115"
